In [1]:
# Options
addRawFeat = True
base_path = ''
feature_networks_integration = ['cna','exp','mut']
#feature_networks_integration = [ 'exp']
#feature_networks_integration = [ 'exp']
node_networks = ['cna','exp','mut']
#node_networks = ['exp']
int_method = 'MLP' # 'MLP' or 'XGBoost' or 'RF' or 'SVM'
xtimes = 50 
xtimes2 = 10 

In [2]:
feature_selection_per_network = [False]*len(feature_networks_integration)
top_features_per_network = [50, 50,50]
optional_feat_selection = False
boruta_runs = 100
boruta_top_features = 50

max_epochs = 500
min_epochs = 200
patience = 30
learning_rates = [0.01, 0.001, 0.0001]
#learning_rates = [0.0001]
# hid_sizes = [16, 32, 64, 128, 256, 512] 
hid_sizes = [512] 
random_state = 404

In [3]:
# MOGAT run
print('MOGAT is setting up!')
from lib import module2, function
import time
import os, pyreadr, itertools
import pickle as pickle#改
from sklearn.metrics import f1_score, accuracy_score
import statistics
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split, RandomizedSearchCV, GridSearchCV
import pandas as pd
import numpy as np
from torch_geometric.data import Data
import os
import torch
import argparse
import errno
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

MOGAT is setting up!


d:\Miniconda3\envs\deep\lib\site-packages\torch_cluster\nearest.py:3: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.23.2)
  import scipy.cluster


In [4]:
if ((True in feature_selection_per_network) or (optional_feat_selection == True)):
    import rpy2
    import rpy2.robjects as robjects
    from rpy2.robjects.packages import importr
    utils = importr('utils')
    rFerns = importr('rFerns')
    Boruta = importr('Boruta')
    pracma = importr('pracma')
    dplyr = importr('dplyr')
    import re

In [5]:
# Parser
parser = argparse.ArgumentParser(description='''An integrative node classification framework, called MOGAT 
(a cancer subtype prediction methodology), that utilizes graph attentions on multiple datatype-specific networks that are annotated with multiomics datasets as node features. 
This framework is model-agnostic and could be applied to any classification problem with properly processed datatypes and networks.
In our work, MOGAT was applied specifically to the breast cancer subtype prediction problem by applying attentions on patient similarity networks
constructed based on multiple biological datasets from breast tumor samples.''')
parser.add_argument('-data', "--data_location", nargs = 1, default = ['data(li)delete10/训练'])
parser.add_argument("--verbosity", help="increase output verbosity")
args = parser.parse_args(args=[])
dataset_name = args.data_location[0]

path = base_path  + dataset_name
if not os.path.exists(path):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), path)
        
device = torch.device('cpu')  # 将设备设置为 CPU
torch.set_default_tensor_type('torch.FloatTensor')  # 设置默认张量类型为 CPU 浮点张量


In [6]:
print(dataset_name)

data(li)delete10/训练


In [7]:
def train():
    model.train()
    optimizer.zero_grad()
    out, emb1 = model(data)
    # print("device",out.get_device(),emb1.get_device(), data.get_device())
    train_idx = data.train_mask
    # train_idx.cpu()
    obj1 = out[train_idx]
    # print(data.y.get_device())
    # print(train_idx.get_device())
    obj2 = data.y[train_idx]
    loss = criterion(obj1, obj2)
    # train_idx.cuda()
    loss.backward()
    optimizer.step()
    return emb1

In [8]:
def validate():
    model.eval()
    with torch.no_grad():
        out, emb2 = model(data)
        pred = out.argmax(dim=1)
        valid_idx = data.valid_mask
        # valid_idx.cpu()
        loss = criterion(out[valid_idx], data.y[valid_idx])
        # valid_idx.cuda()       
    return loss, emb2

In [9]:
criterion = torch.nn.CrossEntropyLoss()

data_path_node =  base_path + dataset_name +'/'
run_name = 'MOGAT_'+  dataset_name + '_results_1'
save_path = base_path + run_name + '/'
excel_file = save_path + "MOGAT_results.xlsx"

if not os.path.exists(base_path + run_name):
    os.makedirs(base_path + run_name + '/')


In [10]:
# 读取 CSV 文件
file = base_path + dataset_name + '/labels.csv'  # 假设您的标签文件为 labels.csv
print("Reading:", file)
# 使用 pandas 读取 CSV 文件
labels_df = pd.read_csv(file)
# 假设标签在 CSV 文件中的一列名为 'label'，您可以根据实际情况调整列名
labels = labels_df['diagnosis'].values  # 将标签转换为 NumPy 数组
#file = base_path + dataset_name +'/labels.pkl'
#print("Reading:", file)
#with open(file, 'rb') as f:
    #labels = pickle.load(f)

file = base_path+ dataset_name + '/mask_values.pkl'
if os.path.exists(file):
    with open(file, 'rb') as f:
        train_valid_idx, test_idx = pickle.load(f)
else:
    train_valid_idx, test_idx= train_test_split(np.arange(len(labels)), test_size=0.20, shuffle=True, stratify=labels, random_state=random_state)

start = time.time()

is_first = 0

print('MOGAT is running..')

Reading: data(li)delete10/训练/labels.csv
MOGAT is running..


In [11]:
for netw in node_networks:
    file = base_path + dataset_name + '/' + netw + '.csv'  # 假设您的特征文件为 netw.csv
    print("Reading:", file)
    # 使用 pandas 读取 CSV 文件
    feat_df = pd.read_csv(file)
    # 如果需要将 DataFrame 转换为 NumPy 数组，可以使用 .values 或 .to_numpy()
    feat = feat_df.values  # 将特征转换为 NumPy 数组
    #file = base_path + 'data/' + dataset_name +'/'+ netw +'.pkl'
    #print("Reading:",file)
    #with open(file, 'rb') as f:
    #feat = pickle.load(f)
    if feature_selection_per_network[node_networks.index(netw)] and top_features_per_network[node_networks.index(netw)] < feat.values.shape[1]:     
        feat_flat = [item for sublist in feat.values.tolist() for item in sublist]
        feat_temp = robjects.FloatVector(feat_flat)
        robjects.globalenv['feat_matrix'] = robjects.r('matrix')(feat_temp)
        robjects.globalenv['feat_x'] = robjects.IntVector(feat.shape)
        robjects.globalenv['labels_vector'] = robjects.IntVector(labels.tolist())
        robjects.globalenv['top'] = top_features_per_network[node_networks.index(netw)]
        robjects.globalenv['maxBorutaRuns'] = boruta_runs
        robjects.r('''
            require(rFerns)
            require(Boruta)
            labels_vector = as.factor(labels_vector)
            feat_matrix <- Reshape(feat_matrix, feat_x[1])
            feat_data = data.frame(feat_matrix)
            colnames(feat_data) <- 1:feat_x[2]
            feat_data <- feat_data %>%
                mutate('Labels' = labels_vector)
            boruta.train <- Boruta(feat_data$Labels ~ ., data= feat_data, doTrace = 0, getImp=getImpFerns, holdHistory = T, maxRuns = maxBorutaRuns)
            thr = sort(attStats(boruta.train)$medianImp, decreasing = T)[top]
            boruta_signif = rownames(attStats(boruta.train)[attStats(boruta.train)$medianImp >= thr,])
                ''')
        boruta_signif = robjects.globalenv['boruta_signif']
        robjects.r.rm("feat_matrix")
        robjects.r.rm("labels_vector")
        robjects.r.rm("feat_data")
        robjects.r.rm("boruta_signif")
        robjects.r.rm("thr")
        topx = []
        for index in boruta_signif:
            t_index=re.sub("`","",index)
            topx.append((np.array(feat.values).T)[int(t_index)-1])
        topx = np.array(topx)
        values = torch.tensor(topx.T, device=device)
    elif feature_selection_per_network[node_networks.index(netw)] and top_features_per_network[node_networks.index(netw)] >= feat.values.shape[1]:
        values = feat
    else:
        values = feat

if is_first == 0:
    new_x = torch.tensor(values, device=device).float()
    is_first = 1
else:
    new_x = torch.cat((new_x, torch.tensor(values, device=device).float()), dim=1)

Reading: data(li)delete10/训练/cna.csv


FileNotFoundError: [Errno 2] No such file or directory: 'data(li)delete10/训练/cna.csv'

In [12]:
for n in range(len(node_networks)):
    netw_base = node_networks[n]
    # 假设 netw_base 是网络名称，data_path_node 是数据路径
    csv_file_path = data_path_node + 'edges_' + netw_base + '.csv'

    # 读取 CSV 文件
    print("Reading", csv_file_path)
    edge_index = pd.read_csv(csv_file_path)

    #with open(data_path_node + 'edges_' + netw_base + '.pkl', 'rb') as f:
        #print("Reading",data_path_node + 'edges_' + netw_base + '.pkl' )
        #edge_index = pickle.load(f)
    best_ValidLoss = np.Inf

    for learning_rate in learning_rates:
        for hid_size in hid_sizes:
            av_valid_losses = list()

            for ii in range(xtimes2):
                data = Data(x=new_x, edge_index=torch.tensor(edge_index[edge_index.columns[0:2]].transpose().values, device=device).long(),
                            edge_attr=torch.tensor(edge_index[edge_index.columns[2]].transpose().values, device=device).float(), y=labels) 
                #data.cuda()
                if isinstance(data.y, np.ndarray):
                  data.y = torch.tensor(data.y, device=device)
                  data.x = torch.tensor(data.x, device=device)
                X = data.x[train_valid_idx]
                y = data.y[train_valid_idx]
                rskf = RepeatedStratifiedKFold(n_splits=4, n_repeats=1)

                for train_part, valid_part in rskf.split(X, y):
                    train_idx = train_valid_idx[train_part]
                    valid_idx = train_valid_idx[valid_part]
                    break

                train_mask = np.array([i in set(train_idx) for i in range(data.x.shape[0])])
                valid_mask = np.array([i in set(valid_idx) for i in range(data.x.shape[0])])
                data.valid_mask = torch.tensor(valid_mask, device=device)
                data.train_mask = torch.tensor(train_mask, device=device)
                test_mask = np.array([i in set(test_idx) for i in range(data.x.shape[0])])
                data.test_mask = torch.tensor(test_mask, device=device)

                in_size = data.x.shape[1]
                out_size = torch.unique(data.y).shape[0]

                print("GAT trained for hyperparameter: learning rate", learning_rate, "hidden layer size", hid_size)
                model = module2.Net(in_size=in_size, hid_size=hid_size, out_size=out_size)
                optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

                min_valid_loss = np.Inf
                patience_count = 0
                data.cpu()
                for epoch in range(max_epochs):
                    emb = train()
                    this_valid_loss, emb = validate()
                    # print("Epoch:",epoch, "val_loss",this_valid_loss)

                    if this_valid_loss < min_valid_loss:
                        min_valid_loss = this_valid_loss
                        patience_count = 0
                    else:
                        patience_count += 1

                    if epoch >= min_epochs and patience_count >= patience:
                        break

                av_valid_losses.append(min_valid_loss.item())

            av_valid_loss = round(statistics.median(av_valid_losses), 3)
            
            if av_valid_loss < best_ValidLoss:
                best_ValidLoss = av_valid_loss
                best_emb_lr = learning_rate
                best_emb_hs = hid_size
                
    data = Data(x=new_x, edge_index=torch.tensor(edge_index[edge_index.columns[0:2]].transpose().values, device=device).long(),
                edge_attr=torch.tensor(edge_index[edge_index.columns[2]].transpose().values, device=device).float(), y=labels) 
    data.cpu()
        # 确保 data.y 是一个 PyTorch 张量
    if isinstance(data.y, np.ndarray):
        data.y = torch.tensor(data.y, device=device)
        # 确保 data.y 是一个 PyTorch 张量
    if isinstance(data.y, np.ndarray):
        data.x = torch.tensor(data.x, device=device)

    X = data.x[train_valid_idx]
    y = data.y[train_valid_idx]
    
    train_mask = np.array([i in set(train_valid_idx) for i in range(data.x.shape[0])])
    data.train_mask = torch.tensor(train_mask, device=device)
    valid_mask = np.array([i in set(test_idx) for i in range(data.x.shape[0])])
    data.valid_mask = torch.tensor(valid_mask, device=device)
    
    in_size = data.x.shape[1]
    out_size = torch.unique(data.y).shape[0]
    
    # initializing GAT
    print("GAT training Started 2")
    model = module2.Net(in_size=in_size, hid_size=best_emb_hs, out_size=out_size)
    optimizer = torch.optim.Adam(model.parameters(), lr=best_emb_lr)

    min_valid_loss = np.Inf
    patience_count = 0
                
    for epoch in range(max_epochs):
        emb = train()
        this_valid_loss, emb = validate()
        # print("Epoch:",epoch, "val_loss",this_valid_loss)

        if this_valid_loss < min_valid_loss:
            min_valid_loss = this_valid_loss
            patience_count = 0
            selected_emb = emb
        else:
            patience_count += 1

        if epoch >= min_epochs and patience_count >= patience:
            break


    emb_file = save_path + 'Emb_' +  netw_base + '.pkl'
    with open(emb_file, 'wb') as f:
        pickle.dump(selected_emb, f)
        pd.DataFrame(selected_emb).to_csv(emb_file[:-4] + '.csv')
    print("GAT training done for", data_path_node + 'edges_' + netw_base + '.pkl')

Reading data(CRC)/训练/edges_path.csv
GAT trained for hyperparameter: learning rate 0.01 hidden layer size 512
GAT trained for hyperparameter: learning rate 0.01 hidden layer size 512
GAT trained for hyperparameter: learning rate 0.01 hidden layer size 512
GAT trained for hyperparameter: learning rate 0.01 hidden layer size 512
GAT trained for hyperparameter: learning rate 0.01 hidden layer size 512
GAT trained for hyperparameter: learning rate 0.01 hidden layer size 512
GAT trained for hyperparameter: learning rate 0.01 hidden layer size 512
GAT trained for hyperparameter: learning rate 0.01 hidden layer size 512
GAT trained for hyperparameter: learning rate 0.01 hidden layer size 512
GAT trained for hyperparameter: learning rate 0.01 hidden layer size 512
GAT trained for hyperparameter: learning rate 0.001 hidden layer size 512
GAT trained for hyperparameter: learning rate 0.001 hidden layer size 512
GAT trained for hyperparameter: learning rate 0.001 hidden layer size 512
GAT trained 

In [13]:
end = time.time()
print('It took ' + str(round(end - start, 1)) + ' seconds for all runs.')
print('MOGAT is done.')
print('Results are available at ' + excel_file)

It took 474.3 seconds for all runs.
MOGAT is done.
Results are available at MOGAT_data(CRC)/训练_results_1/MOGAT_results.xlsx


In [14]:
# 保存模型
model_file_path = "data(CRC)/训练/feature_model.pth"  # 指定保存路径
torch.save(model.state_dict(), model_file_path)
torch.save(model,"data(CRC)/训练/feature_model.model")  # 保存模型的状态字典
print("Model saved to", model_file_path)



Model saved to data(CRC)/训练/feature_model.pth


In [15]:
dataset_name="data(CRC)/外部测试"
# 读取 CSV 文件
file = base_path + dataset_name + '/labels.csv'  # 假设您的标签文件为 labels.csv
print("Reading:", file)
# 使用 pandas 读取 CSV 文件
labels_df = pd.read_csv(file)
# 假设标签在 CSV 文件中的一列名为 'label'，您可以根据实际情况调整列名
labels = labels_df['diagnosis'].values  # 将标签转换为 NumPy 数组
#file = base_path + dataset_name +'/labels.pkl'
#print("Reading:", file)
#with open(file, 'rb') as f:
    #labels = pickle.load(f)
start = time.time()

is_first = 0

print('MOGAT is running..')

Reading: data(CRC)/外部测试/labels.csv
MOGAT is running..


In [16]:

for netw in node_networks:
    file = base_path + dataset_name + '/' + netw + '.csv'  # 假设您的特征文件为 netw.csv
    print("Reading:", file)
    # 使用 pandas 读取 CSV 文件
    feat_df = pd.read_csv(file)
    # 如果需要将 DataFrame 转换为 NumPy 数组，可以使用 .values 或 .to_numpy()
    feat = feat_df.values  # 将特征转换为 NumPy 数组
    #file = base_path + 'data/' + dataset_name +'/'+ netw +'.pkl'
    #print("Reading:",file)
    #with open(file, 'rb') as f:
    #feat = pickle.load(f)
    if feature_selection_per_network[node_networks.index(netw)] and top_features_per_network[node_networks.index(netw)] < feat.values.shape[1]:     
        feat_flat = [item for sublist in feat.values.tolist() for item in sublist]
        feat_temp = robjects.FloatVector(feat_flat)
        robjects.globalenv['feat_matrix'] = robjects.r('matrix')(feat_temp)
        robjects.globalenv['feat_x'] = robjects.IntVector(feat.shape)
        robjects.globalenv['labels_vector'] = robjects.IntVector(labels.tolist())
        robjects.globalenv['top'] = top_features_per_network[node_networks.index(netw)]
        robjects.globalenv['maxBorutaRuns'] = boruta_runs
        robjects.r('''
            require(rFerns)
            require(Boruta)
            labels_vector = as.factor(labels_vector)
            feat_matrix <- Reshape(feat_matrix, feat_x[1])
            feat_data = data.frame(feat_matrix)
            colnames(feat_data) <- 1:feat_x[2]
            feat_data <- feat_data %>%
                mutate('Labels' = labels_vector)
            boruta.train <- Boruta(feat_data$Labels ~ ., data= feat_data, doTrace = 0, getImp=getImpFerns, holdHistory = T, maxRuns = maxBorutaRuns)
            thr = sort(attStats(boruta.train)$medianImp, decreasing = T)[top]
            boruta_signif = rownames(attStats(boruta.train)[attStats(boruta.train)$medianImp >= thr,])
                ''')
        boruta_signif = robjects.globalenv['boruta_signif']
        robjects.r.rm("feat_matrix")
        robjects.r.rm("labels_vector")
        robjects.r.rm("feat_data")
        robjects.r.rm("boruta_signif")
        robjects.r.rm("thr")
        topx = []
        for index in boruta_signif:
            t_index=re.sub("`","",index)
            topx.append((np.array(feat.values).T)[int(t_index)-1])
        topx = np.array(topx)
        values = torch.tensor(topx.T, device=device)
    elif feature_selection_per_network[node_networks.index(netw)] and top_features_per_network[node_networks.index(netw)] >= feat.values.shape[1]:
        values = feat
    else:
        values = feat

if is_first == 0:
    new_x = torch.tensor(values, device=device).float()
    is_first = 1
else:
    new_x = torch.cat((new_x, torch.tensor(values, device=device).float()), dim=1)

Reading: data(CRC)/外部测试/path.csv
Reading: data(CRC)/外部测试/cell.csv


In [18]:
for n in range(len(node_networks)):
    netw_base = node_networks[n]
    csv_file_path = dataset_name +'/'+ 'edges_' + netw_base + '.csv'

    # 读取 CSV 文件
    print("Reading", csv_file_path)
    edge_index = pd.read_csv(csv_file_path)

    # 准备数据
    data = Data(
        x=new_x,
        edge_index=torch.tensor(edge_index[edge_index.columns[0:2]].transpose().values, device=device).long(),
        edge_attr=torch.tensor(edge_index[edge_index.columns[2]].transpose().values, device=device).float(),
        y=labels
    )

    # 确保 data.y 是一个 PyTorch 张量
    if isinstance(data.y, np.ndarray):
        data.y = torch.tensor(data.y, device=device)

    # 创建测试集掩码
    test_mask = np.array([i in set(test_idx) for i in range(data.x.shape[0])])
    data.test_mask = torch.tensor(test_mask, device=device)

    # 这里假设您已经有一个训练好的模型
    # 加载模型
    in_size = data.x.shape[1]
    out_size = torch.unique(data.y).shape[0]
    model_file = "data(CRC)/训练/feature_model.pth"  # 假设您保存了模型
    model = module2.Net(in_size=in_size, hid_size=hid_size, out_size=out_size)

    model.load_state_dict(torch.load(model_file))
    model.eval()  # 设置模型为评估模式

    # 在你的代码中找到处理模型输出的部分
    with torch.no_grad():  # 不需要计算梯度
        out = model(data)  # 仅对测试集进行预测

    # 检查模型输出的类型
    if isinstance(out, tuple):
        # 如果输出是一个元组，选择你需要的部分
        # 例如，如果你只需要第一个输出，可以这样做：
        out = out[1]  # 选择第一个元素，假设这是 logits
    output_file = save_path + 'Output_' + netw_base + '.pkl'
    # 然后继续处理输出
    out_np = out.detach().cpu().numpy()  # 将张量从 GPU 移动到 CPU 并转换为 NumPy 数组
    pd.DataFrame(out_np).to_csv(output_file[:-4] + '.csv', index=False)

    print("GAT training done for", '外部测试' + '/' + 'edges_' + netw_base + '.pkl')
    print("Output saved to", output_file)




Reading data(CRC)/外部测试/edges_path.csv
GAT training done for 外部测试/edges_path.pkl
Output saved to MOGAT_data(CRC)/训练_results_1/Output_path.pkl
Reading data(CRC)/外部测试/edges_cell.csv
GAT training done for 外部测试/edges_cell.pkl
Output saved to MOGAT_data(CRC)/训练_results_1/Output_cell.pkl


In [19]:
# Options
addRawFeat = True
base_path = ''
feature_networks_integration = [ 'path','cell']
#feature_networks_integration = [ 'exp']
node_networks = [ 'path','cell']
#node_networks = [ 'exp']
int_method = 'MLP' # 'MLP' or 'XGBoost' or 'RF' or 'SVM'
xtimes = 50 
xtimes2 = 10 

feature_selection_per_network = [False]*len(feature_networks_integration)
top_features_per_network = [50, 50]
optional_feat_selection = False
boruta_runs = 100
boruta_top_features = 50

max_epochs = 500
min_epochs = 200
patience = 30
learning_rates = [0.01, 0.001, 0.0001]
#learning_rates = [0.0001]
# hid_sizes = [16, 32, 64, 128, 256, 512] 
hid_sizes = [512] 
random_state = 404

In [20]:
# MOGAT run
print('MOGAT is setting up!')
from lib import function
import time, io
import os, pyreadr, itertools
import pickle as pickle
from sklearn.metrics import f1_score, accuracy_score
import statistics
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split, RandomizedSearchCV, GridSearchCV
import pandas as pd
import numpy as np
import os
import torch
import argparse
import errno
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


MOGAT is setting up!


In [21]:
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

class Data:
    def __init__(self, x=None, y=None):
        self.x=x
        self.y=y

    def cpu(self):
        self.x = self.x.cpu()
        self.y = self.y.cpu()

if ((True in feature_selection_per_network) or (optional_feat_selection == True)):
    import rpy2
    import rpy2.robjects as robjects
    from rpy2.robjects.packages import importr
    utils = importr('utils')
    rFerns = importr('rFerns')
    Boruta = importr('Boruta')
    pracma = importr('pracma')
    dplyr = importr('dplyr')
    import re

In [22]:

path = base_path  + dataset_name
if not os.path.exists(path):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), path)
        
device = torch.device('cpu')
#torch.set_default_tensor_type('torch.cuda.FloatTensor')
#torch.cuda.set_device(6)


data_path_node =  base_path + dataset_name +'/'
run_name = 'MOGAT_'+  dataset_name + '_results_1'
save_path = base_path + run_name + '/'
excel_file = save_path + "MOGAT_results.xlsx"

if not os.path.exists(base_path + run_name):
    os.makedirs(base_path + run_name + '/')

# 读取 CSV 文件
file = base_path + dataset_name + '/labels.csv'  # 假设您的标签文件为 labels.csv
print("Reading:", file)
# 使用 pandas 读取 CSV 文件
labels_df = pd.read_csv(file)
# 假设标签在 CSV 文件中的一列名为 'label'，您可以根据实际情况调整列名
labels = labels_df['diagnosis'].values  # 将标签转换为 NumPy 数组

file = base_path + 'data/' + dataset_name + '/mask_values.pkl'
if os.path.exists(file):
    with open(file, 'rb') as f:
        train_valid_idx, test_idx = pickle.load(f)
else:
    train_valid_idx, test_idx= train_test_split(np.arange(len(labels)), test_size=0.20, shuffle=True, stratify=labels, random_state=random_state)

start = time.time()

is_first = 0

print('MOGAT is running..')


Reading: 训练/labels.csv
MOGAT is running..


In [3]:
addFeatures = []
t = range(len(node_networks))
trial_combs = []
for r in range(1, len(t) + 1):
    trial_combs.extend([list(x) for x in itertools.combinations(t, r)])
new_trial_combs = []
for set1 in trial_combs:
    new_trial_combs.append(list(set1))
trial_combs = new_trial_combs

device = torch.device('cpu')


NameError: name 'node_networks' is not defined

In [2]:
print(trial_combs)
range(len(trial_combs))

NameError: name 'trial_combs' is not defined

In [25]:
for trials in range(len(trial_combs)):
    node_networks2 = [node_networks[i] for i in trial_combs[trials]] # list(set(a) & set(feature_networks))
    netw_base = node_networks2[0]
    emb_file = save_path + 'Emb_' +  netw_base + '.pkl'
    with open(emb_file, 'rb') as f:
        #emb = pickle.load(f)
        emb = CPU_Unpickler(f).load()
    emb = emb.cpu()
    if len(node_networks2) > 1:
        for netw_base in node_networks2[1:]:
            emb_file = save_path + 'Emb_' +  netw_base + '.pkl'
            with open(emb_file, 'rb') as f:
                #emb = pickle.load(f)
                cur_emb = CPU_Unpickler(f).load()
                cur_emb = cur_emb.cpu()
            emb = torch.cat((emb, cur_emb), dim=1)
    emb = emb.cpu()        
    if addRawFeat == True:
        is_first = 0
        addFeatures = feature_networks_integration
        for netw in addFeatures:
            file = base_path + dataset_name + '/' + netw + '.csv'  # 假设您的特征文件为 netw.csv
            print("Reading:", file)
            # 使用 pandas 读取 CSV 文件
            feat_df = pd.read_csv(file)
            # 如果需要将 DataFrame 转换为 NumPy 数组，可以使用 .values 或 .to_numpy()
            feat = feat_df.values  # 将特征转换为 NumPy 数组
            if is_first == 0:
                allx = torch.tensor(feat, device=device).float()
                is_first = 1
            else:
                allx = torch.cat((allx, torch.tensor(feat, device=device).float()), dim=1)   
        
        if optional_feat_selection == True:     
            allx_flat = [item for sublist in allx.tolist() for item in sublist]
            allx_temp = robjects.FloatVector(allx_flat)
            robjects.globalenv['allx_matrix'] = robjects.r('matrix')(allx_temp)
            robjects.globalenv['allx_x'] = robjects.IntVector(allx.shape)
            robjects.globalenv['labels_vector'] = robjects.IntVector(labels.tolist())
            robjects.globalenv['top'] = boruta_top_features
            robjects.globalenv['maxBorutaRuns'] = boruta_runs
            robjects.r('''
                require(rFerns)
                require(Boruta)
                labels_vector = as.factor(labels_vector)
                allx_matrix <- Reshape(allx_matrix, allx_x[1])
                allx_data = data.frame(allx_matrix)
                colnames(allx_data) <- 1:allx_x[2]
                allx_data <- allx_data %>%
                    mutate('Labels' = labels_vector)
                boruta.train <- Boruta(allx_data$Labels ~ ., data= allx_data, doTrace = 0, getImp=getImpFerns, holdHistory = T, maxRuns = maxBorutaRuns)
                thr = sort(attStats(boruta.train)$medianImp, decreasing = T)[top]
                boruta_signif = rownames(attStats(boruta.train)[attStats(boruta.train)$medianImp >= thr,])
                    ''')
            boruta_signif = robjects.globalenv['boruta_signif']
            robjects.r.rm("allx_matrix")
            robjects.r.rm("labels_vector")
            robjects.r.rm("allx_data")
            robjects.r.rm("boruta_signif")
            robjects.r.rm("thr")
            topx = []
            for index in boruta_signif:
                t_index=re.sub("`","",index)
                topx.append((np.array(allx).T)[int(t_index)-1])
            topx = np.array(topx)
            emb = torch.cat((emb, torch.tensor(topx.T, device=device)), dim=1)
            print('Top ' + str(boruta_top_features) + " features have been selected.")
        else:
            print(emb.get_device())
            print(allx.get_device())
            emb = torch.cat((emb, allx), dim=1)
    labels_tensor = torch.tensor(labels, device=device)  # 将 labels 转换为张量

    data = Data(x=emb, y=labels_tensor)  # 使用张量创建 Data 对象

    #data = Data(x=emb, y=labels)
    
    data.cpu()
    train_mask = np.array([i in set(train_valid_idx) for i in range(data.x.shape[0])])
    data.train_mask = torch.tensor(train_mask, device=device)
    test_mask = np.array([i in set(test_idx) for i in range(data.x.shape[0])])
    data.test_mask = torch.tensor(test_mask, device=device)
    X_train = pd.DataFrame(data.x[data.train_mask].numpy())
    X_test = pd.DataFrame(data.x[data.test_mask].numpy())
    y_train = pd.DataFrame(data.y[data.train_mask].numpy()).values.ravel()
    y_test = pd.DataFrame(data.y[data.test_mask].numpy()).values.ravel()
    print("Second Model Training Started")

    if int_method == 'MLP':
        params = {'hidden_layer_sizes': [ (64, 32)],
                  'learning_rate_init': [0.001],
                  'max_iter': [ 1500],
                  'n_iter_no_change': [100]}
        search = RandomizedSearchCV(estimator = MLPClassifier(solver = 'adam', activation = 'relu', early_stopping = True), 
                                    return_train_score = True, scoring = 'f1_macro', 
                                    param_distributions = params, cv = 4, n_iter = xtimes, verbose = 0)
        search.fit(X_train, y_train)
        model = MLPClassifier(solver = 'adam', activation = 'relu', early_stopping = True,
                              max_iter = search.best_params_['max_iter'], 
                              n_iter_no_change = search.best_params_['n_iter_no_change'],
                              hidden_layer_sizes = search.best_params_['hidden_layer_sizes'],
                              learning_rate_init = search.best_params_['learning_rate_init'])
        
    elif int_method == 'XGBoost':
        params = {'reg_alpha':range(0,10,1), 'reg_lambda':range(1,10,1) ,'max_depth': range(1,6,1), 
                  'min_child_weight': range(1,10,1), 'gamma': range(0,6,1),
                  'learning_rate':[0, 1e-5, 0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 1],
                  'max_delta_step': range(0,10,1), 'colsample_bytree': [0.5, 0.7, 1.0],
                  'colsample_bylevel': [0.5, 0.7, 1.0], 'colsample_bynode': [0.5, 0.7, 1.0]}
        fit_params = {'early_stopping_rounds': 10,
                     'eval_metric': 'mlogloss',
                     'eval_set': [(X_train, y_train)]}
        
              
        search = RandomizedSearchCV(estimator = XGBClassifier(use_label_encoder=False, n_estimators = 1000, 
                                                                  fit_params = fit_params, objective="multi:softprob", eval_metric = "mlogloss", 
                                                                  verbosity = 0), return_train_score = True, scoring = 'f1_macro',
                                        param_distributions = params, cv = 4, n_iter = xtimes, verbose = 0)
        
        search.fit(X_train, y_train)
        
        model = XGBClassifier(use_label_encoder=False, objective="multi:softprob", eval_metric = "mlogloss", verbosity = 0,
                              n_estimators = 1000, fit_params = fit_params,
                              reg_alpha = search.best_params_['reg_alpha'],
                              reg_lambda = search.best_params_['reg_lambda'],
                              max_depth = search.best_params_['max_depth'],
                              min_child_weight = search.best_params_['min_child_weight'],
                              gamma = search.best_params_['gamma'],
                              learning_rate = search.best_params_['learning_rate'],
                              max_delta_step = search.best_params_['max_delta_step'],
                              colsample_bytree = search.best_params_['colsample_bytree'],
                              colsample_bylevel = search.best_params_['colsample_bylevel'],
                              colsample_bynode = search.best_params_['colsample_bynode'])
                            
    elif int_method == 'RF':
        max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
        max_depth.append(None)
        params = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
                  'max_depth': max_depth,
                  'min_samples_split': [2, 5, 7, 10],
                  'min_samples_leaf': [1, 2, 5, 7, 10], 
                 'min_impurity_decrease':[0,0.5, 0.7, 1, 5, 10],
                 'max_leaf_nodes': [None, 5, 10, 20]}
        
        search = RandomizedSearchCV(estimator = RandomForestClassifier(), return_train_score = True,
                                    scoring = 'f1_macro', param_distributions = params, cv=4,  n_iter = xtimes, verbose = 0)
        
        search.fit(X_train, y_train)
        model=RandomForestClassifier(n_estimators = search.best_params_['n_estimators'],
                                     max_depth = search.best_params_['max_depth'],
                                     min_samples_split = search.best_params_['min_samples_split'],
                                     min_samples_leaf = search.best_params_['min_samples_leaf'],
                                     min_impurity_decrease = search.best_params_['min_impurity_decrease'],
                                     max_leaf_nodes = search.best_params_['max_leaf_nodes'])

    elif int_method == 'SVM':
        params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                  'gamma': [1, 0.1, 0.01, 0.001, 0.0001, 'scale', 'auto'],
                  'kernel': ['linear', 'rbf']}
        
        search = RandomizedSearchCV(SVC(), return_train_score = True,
                                    scoring = 'f1_macro', param_distributions = params, cv=4, n_iter = xtimes, verbose = 0)
        
        search.fit(X_train, y_train)
        model = SVC(kernel=search.best_params_['kernel'],
                  C = search.best_params_['C'],
                  gamma = search.best_params_['gamma'])

 
    av_result_acc = list()
    av_result_wf1 = list()
    av_result_mf1 = list()
    av_tr_result_acc = list()
    av_tr_result_wf1 = list()
    av_tr_result_mf1 = list()
 
        
    for ii in range(xtimes2):
        model.fit(X_train,y_train)
        predictions = model.predict(X_test)
        y_pred = [round(value) for value in predictions]
        preds = model.predict(pd.DataFrame(data.x.numpy()))
        av_result_acc.append(round(accuracy_score(y_test, y_pred), 3))
        av_result_wf1.append(round(f1_score(y_test, y_pred, average='weighted'), 3))
        av_result_mf1.append(round(f1_score(y_test, y_pred, average='macro'), 3))
        tr_predictions = model.predict(X_train)
        tr_pred = [round(value) for value in tr_predictions]
        av_tr_result_acc.append(round(accuracy_score(y_train, tr_pred), 3))
        av_tr_result_wf1.append(round(f1_score(y_train, tr_pred, average='weighted'), 3))
        av_tr_result_mf1.append(round(f1_score(y_train, tr_pred, average='macro'), 3))
        
    if xtimes2 == 1:
        av_result_acc.append(round(accuracy_score(y_test, y_pred), 3))
        av_result_wf1.append(round(f1_score(y_test, y_pred, average='weighted'), 3))
        av_result_mf1.append(round(f1_score(y_test, y_pred, average='macro'), 3))
        av_tr_result_acc.append(round(accuracy_score(y_train, tr_pred), 3))
        av_tr_result_wf1.append(round(f1_score(y_train, tr_pred, average='weighted'), 3))
        av_tr_result_mf1.append(round(f1_score(y_train, tr_pred, average='macro'), 3))
        

    result_acc = str(round(statistics.median(av_result_acc), 3)) + '+-' + str(round(statistics.stdev(av_result_acc), 3))
    result_wf1 = str(round(statistics.median(av_result_wf1), 3)) + '+-' + str(round(statistics.stdev(av_result_wf1), 3))
    result_mf1 = str(round(statistics.median(av_result_mf1), 3)) + '+-' + str(round(statistics.stdev(av_result_mf1), 3))
    tr_result_acc = str(round(statistics.median(av_tr_result_acc), 3)) + '+-' + str(round(statistics.stdev(av_tr_result_acc), 3))
    tr_result_wf1 = str(round(statistics.median(av_tr_result_wf1), 3)) + '+-' + str(round(statistics.stdev(av_tr_result_wf1), 3))
    tr_result_mf1 = str(round(statistics.median(av_tr_result_mf1), 3)) + '+-' + str(round(statistics.stdev(av_tr_result_mf1), 3))
    
    
    # 创建一个空的 DataFrame，只需在外部定义一次
    if 'df' not in locals():  # 检查 df 是否已经定义
        df = pd.DataFrame(columns=['Comb No', 'Used Embeddings', 'Added Raw Features', 'Selected Params', 'Train Acc', 'Train wF1', 'Train mF1', 'Test Acc', 'Test wF1', 'Test mF1'])

    # 创建一个新的 Series
    x = [trials, node_networks2, addFeatures, search.best_params_, 
        tr_result_acc, tr_result_wf1, tr_result_mf1, result_acc, result_wf1, result_mf1]
    new_row = pd.Series(x, index=df.columns)

    # 使用 pd.concat() 添加新行
    df = pd.concat([df, new_row.to_frame().T], ignore_index=True)
    
    print('Combination ' + str(trials) + ' ' + str(node_networks2) + ' >  selected parameters = ' + str(search.best_params_) + 
      ', train accuracy = ' + str(tr_result_acc) + ', train weighted-f1 = ' + str(tr_result_wf1) +
      ', train macro-f1 = ' +str(tr_result_mf1) + ', test accuracy = ' + str(result_acc) + 
      ', test weighted-f1 = ' + str(result_wf1) +', test macro-f1 = ' +str(result_mf1))


Reading: 训练/path.csv
Reading: 训练/cell.csv
-1
-1
Second Model Training Started
Combination 0 ['path'] >  selected parameters = {'n_iter_no_change': 100, 'max_iter': 1500, 'learning_rate_init': 0.001, 'hidden_layer_sizes': (64, 32)}, train accuracy = 0.816+-0.015, train weighted-f1 = 0.815+-0.015, train macro-f1 = 0.798+-0.016, test accuracy = 0.736+-0.016, test weighted-f1 = 0.736+-0.014, test macro-f1 = 0.718+-0.018
Reading: 训练/path.csv
Reading: 训练/cell.csv
-1
-1
Second Model Training Started
Combination 1 ['cell'] >  selected parameters = {'n_iter_no_change': 100, 'max_iter': 1500, 'learning_rate_init': 0.001, 'hidden_layer_sizes': (64, 32)}, train accuracy = 0.864+-0.031, train weighted-f1 = 0.864+-0.033, train macro-f1 = 0.852+-0.038, test accuracy = 0.79+-0.022, test weighted-f1 = 0.786+-0.025, test macro-f1 = 0.761+-0.029
Reading: 训练/path.csv
Reading: 训练/cell.csv
-1
-1
Second Model Training Started
Combination 2 ['path', 'cell'] >  selected parameters = {'n_iter_no_change': 100, '

In [26]:
print(emb)

tensor([[ 0.2847,  0.2172, -0.0086,  ...,  0.1560, -0.0613,  0.1376],
        [ 0.3201,  0.2804, -0.0451,  ...,  0.1914,  0.0169,  0.1848],
        [ 0.2479,  0.2541, -0.0144,  ...,  0.1752,  0.0586,  0.1990],
        ...,
        [ 0.0554,  0.1556, -0.1033,  ...,  0.2042,  0.1142,  0.2015],
        [ 0.0430,  0.1774, -0.1176,  ...,  0.1959,  0.1647,  0.2079],
        [ 0.0896,  0.1983, -0.1619,  ...,  0.1934,  0.1689,  0.2400]])


In [27]:
import pandas as pd

# 假设 test_idx, y_pred 和 X_test 是您的数据
# 将它们转换为 DataFrame
# 如果 test_idx 是一个列表或数组
test_idx_df = pd.DataFrame(test_idx, columns=['Test Index'])

# 如果 y_pred 是一个列表或数组
y_pred_df = pd.DataFrame(y_pred, columns=['Predictions'])

# 如果 X_test 是一个 DataFrame 或者可以转换为 DataFrame
y_test_df = pd.DataFrame(y_test)

# 将所有数据合并到一个 DataFrame 中
result_df = pd.concat([test_idx_df, y_pred_df, y_test_df], axis=1)

# 导出为 CSV 文件
result_df.to_csv('D:\\test_results.csv', index=False)

print("Data exported successfully to test_results.csv")

Data exported successfully to test_results.csv


In [28]:
end = time.time()
print('It took ' + str(round(end - start, 1)) + ' seconds for all runs.')
print('MOGAT is done.')
print('Results are available at ' + excel_file)

It took 182.5 seconds for all runs.
MOGAT is done.
Results are available at MOGAT_训练_results_1/MOGAT_results.xlsx


In [29]:
import joblib  # 或者 import pickle

# 假设 model 是组合2训练后的模型
model_filename = 'model_combination_2.pkl'  # 指定保存的文件名
joblib.dump(model, model_filename)  # 使用 joblib 保存模型
torch.save(model,"model_combination_2.model")

In [50]:
import pandas as pd

# 假设三个 CSV 文件的路径
file1 = '外部测试\\path.csv'
file2 = '外部测试\\cell.csv'
file3 = 'MOGAT_训练_results_1\\Output_path.csv'
file4='MOGAT_训练_results_1\\Output_cell.csv'
# 读取 CSV 文件
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)
df4 = pd.read_csv(file4)
# 将三个 DataFrame 的列整合到一起
X_test = pd.concat([df1, df2, df3,df4], axis=1)
X_test.columns = range(1453)
# 输出整合后的 X_test 的形状和内容
print(f"X_test shape: {X_test.shape}")
print(X_test.head())

X_test shape: (832, 1453)
       0         1         2         3         4         5         6     \
0  0.188567 -0.002946  0.259677  0.232879  0.299379  0.245001  0.315489   
1  0.191661 -0.039185  0.246106  0.213106  0.305355  0.248259  0.299791   
2  0.209665 -0.028581  0.229080  0.210698  0.302300  0.239930  0.289490   
3  0.233885 -0.014957  0.244939  0.205171  0.298337  0.235393  0.289586   
4  0.197028 -0.008709  0.262457  0.210052  0.287729  0.285531  0.319330   

       7         8         9     ...      1443      1444      1445      1446  \
0  0.180616  0.098148 -0.055966  ... -1.129061  0.135429  0.135784 -0.008405   
1  0.147041  0.089811 -0.082131  ... -1.091605  0.185639  0.113645 -0.013304   
2  0.161417  0.079199 -0.077170  ... -1.093004  0.069215  0.133506 -0.041868   
3  0.143058  0.087553 -0.052084  ... -1.051855  0.063913  0.041288 -0.008137   
4  0.149408  0.094126 -0.053295  ... -1.109466  0.178767  0.134429 -0.024522   

       1447      1448      1449      1450 

In [51]:
path = base_path  + dataset_name
if not os.path.exists(path):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), path)
        
device = torch.device('cpu')
#torch.set_default_tensor_type('torch.cuda.FloatTensor')
#torch.cuda.set_device(6)


data_path_node =  base_path + dataset_name +'/'
run_name = 'MOGAT_'+  dataset_name + '_results_1'
save_path = base_path + run_name + '/'
excel_file = save_path + "MOGAT_results.xlsx"

if not os.path.exists(base_path + run_name):
    os.makedirs(base_path + run_name + '/')

# 读取 CSV 文件
file = '外部测试\\labels.csv'  # 假设您的标签文件为 labels.csv
print("Reading:", file)
# 使用 pandas 读取 CSV 文件
labels_df = pd.read_csv(file)
# 假设标签在 CSV 文件中的一列名为 'label'，您可以根据实际情况调整列名
labels = labels_df['diagnosis'].values  # 将标签转换为 NumPy 数组

file = base_path + 'data/' + dataset_name + '/mask_values.pkl'

start = time.time()

is_first = 0

print('MOGAT is running..')

Reading: 外部测试\labels.csv
MOGAT is running..


In [52]:
import joblib
# 2. 加载保存的模型
model_filename = '训练\\model_combination_2.pkl'  # 指定保存的文件名
model = joblib.load(model_filename)

In [53]:
labels_tensor = torch.tensor(labels, device=device)  # 将 labels 转换为张量

data = Data(x=X_test, y=labels_tensor)  # 使用张量创建 Data 对象



In [54]:
X_test = pd.DataFrame(data.x)
y_test = pd.DataFrame(data.y).values.ravel()

In [55]:


# 使用训练好的模型进行预测
predictions_external = model.predict(X_test)



In [60]:

predictions_external = pd.DataFrame(predictions_external)
result_df = pd.concat([labels_df,predictions_external], axis=1)
result_df.to_csv('D:\\test_results.csv', index=False)
